<a href="https://colab.research.google.com/github/pokem1402/section_project_one/blob/main/EDA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Loading Dataset

 EDA_1.ipynb 을 통해 Year, Publisher, Genre의 결손치를 보충하고 결손된 데이터를 제거한 데이터를 불러온다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("/content/drive/My Drive/dataset/vgames2_modified.csv").drop(axis=1, columns="Unnamed: 0")

# 2. 분석할 데이터와 분류 값들을 분리

1. 게임 이름
2. 게임 플랫폼
3. 게임 출시년도
4. 게임 장르
5. 게임 퍼블리셔

6. 게임의 각 지역의 판매량 (북미, 유럽, 일본, 그외 지역)

In [3]:
_name = df.Name
_platform = df.Platform
_year = df.Year.astype(int)
_genre = df.Genre
_publisher = df.Publisher

In [4]:
df_numeric = df.take(range(5,9), axis=1)

In [5]:
try:
  df_numeric.dtype= df_numeric.astype(float)
except BaseException as e:
  print(e)

could not convert string to float: '480K'


# 3. 데이터에 표기 오류값 정정

바로 윗 코드에서 string을 float로 변환하지 못하는 문제가 발생했다. 어떤 항목에서 문제를 일으키는 것인지 확인해보자.

#### 우선 float로 변환 가능한 데이터 row와 불가능한 row를 분리한다.

In [6]:
# 변환 가능한 row들의 집합
valid_index = df_numeric[(df_numeric.replace("[.]", "", regex=True).applymap(lambda x : x.isdigit())).all(1)].index
# 변환 불가능한 row들의 집합
invalid_index = df_numeric[(df_numeric.replace("[.]", "", regex=True).applymap(lambda x : x.isdigit()) == False).any(1)].index
# 두 집합이 서로 독립이면서 합집합이 원래 집합이 되는 것을 확인한다.
df.shape[0] == valid_index.shape[0] + invalid_index.shape[0]

True

In [7]:
# Data check
# 우선 우세적인 수치데이터(K와 M 등의 표기가 따로 없는 데이터)의 최대값과 최소값을 알아보자.
data = df_numeric.replace("K", 0, regex=True).replace("M", 0, regex=True).loc[invalid_index]
data=data.astype(float)

for column in data.columns:
  print(column, data[column].idxmax(), data[column].max(), data[column].idxmin(), data[column].min())
# data

NA_Sales 6452 6.91 10 0.0
EU_Sales 10903 5.31 10 0.0
JP_Sales 577 2.23 10 0.0
Other_Sales 10903 1.38 64 0.0


In [8]:
print(data.NA_Sales.loc[6452], data.NA_Sales.loc[10])
print(data.EU_Sales.loc[10903], data.EU_Sales.loc[10])
print(data.JP_Sales.loc[577], data.JP_Sales.loc[10])
print(data.Other_Sales.loc[10903], data.Other_Sales.loc[64])

6.91 0.0
5.31 0.0
2.23 0.0
1.38 0.0


우선 우세적인 수치 상의 각 데이터는 $[0, 7]$ 사이에 존재하는 것을 확인 할 수 있다. 그러면 이 수치의 스케일이 어떤 의미인지 알아야한다.

 각 항목이 0이라는 같은 최소값을 가지고 있으나 다른 최대값을 가지므로 min-max scaling이나 정규화를 거친 데이터는 아닌 것을 확인할 수 있다.

 그렇다면 각 최댓값을 가지는 수치의 게임의 데이터에 대해 검색하여 이를 확인해보자.

In [9]:
import lxml.html as lh
import requests
import time
import re

def get_data_from_web(game_name, NA_Sales = True,	EU_Sales = True,	JP_Sales =True,	Other_Sales= True, sleep_time = 10, wait_count = 5):
  print(game_name)
  return_value = {"Name":game_name, "Complete" : False, "error": False}
  try:
    game_name_ = game_name.replace(" ", "+")
    base_url = "https://www.vgchartz.com/games/games.php?name="
    search_attr ="&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=50&order=NASales&showtotalsales=0&showpublisher=0&showvgchartzscore=0&shownasales=0&shownasales=1&showdeveloper=0&showcriticscore=0&showpalsales=0&showpalsales=1&showreleasedate=0&showuserscore=0&showjapansales=0&showjapansales=1&showlastupdate=0&showothersales=0&showothersales=1&showshipped=0"
    resp = requests.get(base_url+game_name+search_attr)
    
    count = 0
    while(not resp.ok):
      print("not ok")
      time.sleep(sleep_time)
      resp = requests.get(base_url+game_name+search_attr)
      count+=1
      if(count > wait_count):
        print("Time out")
        return return_value
    doc = lh.fromstring(resp.content)

    def get_str(element):
      s = str(element.text_content())
      s = re.sub(r"(\s\s+)", "", s)
      return s 

    names = doc.xpath('//tr/td[3]')
    # idx = -1;
    idxs = []
    for i,name in enumerate(names[4:]):
      
      name_ = get_str(name)
      extra_string_start = name_.find("Read the review")

      if (extra_string_start != -1):
        name_ = name_[:extra_string_start]
      if(name_ == game_name) or (name_.lower() == game_name.lower()) or (re.sub("[.!]", "", name_.lower()) == re.sub("[.!]", "", game_name.lower())):
        # idx = i
        idxs.append(i)

    if NA_Sales:
      na = doc.xpath('//tr/td[5]')
      na_flag = True
      for idx_ in idxs:
        na_ = get_str(na[idx_+3])
        na_ = re.sub(" ", "", na_)
        if(na_ != "N/A"):
          print(" NA_Sales : ",na_, end="")
          na_flag = False
          break;
      if na_flag:
        print(" NA_Sales : ",0, end="")        

    if EU_Sales:
      eu = doc.xpath('//tr/td[6]')
      eu_flag = True
      for idx_ in idxs:
        eu_ = get_str(eu[idx_+3])
        eu_ = re.sub(" ", "", eu_)
        if(eu_ != "N/A"):
          print(" EU_Sales : ",eu_, end="")
          eu_flag = False
          break
      if eu_flag:
          print(" EU_Sales : ",0, end="")   

    if JP_Sales:
      jp = doc.xpath('//tr/td[7]')
      jp_flag = True
      for idx_ in idxs:
        jp_ = get_str(jp[idx_])
        jp_ = re.sub(" ", "", jp_)
        if(jp_ != "N/A"):
          print(" JP_Sales : ",jp_, end="")
          jp_flag = False
          break
      if jp_flag:
          print(" JP_Sales : ",0, end="")

    if Other_Sales:
      otr = doc.xpath('//tr/td[8]')
      otr_flag = True
      for idx_ in idxs:
        otr_ = get_str(otr[idx_])
        otr_ = re.sub(" ", "", otr_)
        if(otr_ != "N/A"):
          print(" Other_Sales : ",otr_, end="")
          otr_flag = False
          break
      if otr_flag:
        print(" Other_Sales : ",0, end="")
    print("")
  except BaseException as e:
    print(e)

In [10]:
get_data_from_web(_name.loc[6452])
print(data.loc[6452])
get_data_from_web(_name.loc[10])
print(data.loc[10])
get_data_from_web(_name.loc[10903])
print(data.loc[10903])
get_data_from_web(_name.loc[577])
print(data.loc[577])
get_data_from_web(_name.loc[10903])
print(data.loc[10903])
get_data_from_web(_name.loc[64])
print(data.loc[64])


Super Mario 64
 NA_Sales :  6.91m EU_Sales :  2.85m JP_Sales :  1.91m Other_Sales :  0.23m
NA_Sales       6.91
EU_Sales       0.00
JP_Sales       1.91
Other_Sales    0.23
Name: 6452, dtype: float64
Ford Racing
 NA_Sales :  0.48m EU_Sales :  0.33m JP_Sales :  0 Other_Sales :  0.06m
NA_Sales       0.00
EU_Sales       0.00
JP_Sales       0.00
Other_Sales    0.06
Name: 10, dtype: float64
Grand Theft Auto V
 NA_Sales :  9.06m EU_Sales :  5.33m JP_Sales :  0.06m Other_Sales :  1.42m
NA_Sales       0.00
EU_Sales       5.31
JP_Sales       0.06
Other_Sales    1.38
Name: 10903, dtype: float64
Mario Kart 64
 NA_Sales :  5.55m EU_Sales :  1.94m JP_Sales :  2.23m Other_Sales :  0.15m
NA_Sales       0.00
EU_Sales       1.94
JP_Sales       2.23
Other_Sales    0.15
Name: 577, dtype: float64
Grand Theft Auto V
 NA_Sales :  9.06m EU_Sales :  5.33m JP_Sales :  0.06m Other_Sales :  1.42m
NA_Sales       0.00
EU_Sales       5.31
JP_Sales       0.06
Other_Sales    1.38
Name: 10903, dtype: float64
SD Gundam F

데이터가 정확히 숫자가 아닌 데이터를 0으로 만든 까닭에 크롭한 데이터와 데이터가 차이를 보이긴 하지만 기준으로 한 웹 데이터를 보면 데이터가 나타내는 수치는 대략 백만 단위를 기준으로 소숫점 두자리 이하를 반올림한 것으로 보인다.

따라서 데이터를 들여다볼 때 K와 M 이 존재하는데 이는 $10^3$을 나타내는 접두어인 Kilo와 $10^6$을 나타내는 접두어인 Mega일 것이다.

 따라서 k에는 0.001을 곱한 것을, 그리고 M은 1 곱한 결과로로 교체한다.

In [11]:
df

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16578,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16579,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16580,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16581,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


K 값은 즉 $/10^3$을 하는 것과 마찬가지이기 때문에 k 값을 ***e-3***으로 교체하고, M 값은 변화를 일으키지 않기 때문에 지우기만 한 뒤 float로 변환한다.

In [12]:
df_numeric_ = df_numeric.applymap(lambda x : x.replace("K", 'e-3'))
df_numeric_ = df_numeric_.applymap(lambda x : x.replace("M", ''))
df_numeric_ = df_numeric_.astype(float)

In [13]:
df_numeric_

,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,0.04,0.00,0.00,0.00
1,0.17,0.00,0.00,0.01
2,0.00,0.00,0.02,0.00
3,0.04,0.00,0.00,0.00
4,0.12,0.09,0.00,0.04
...,...,...,...,...
16578,0.15,0.04,0.00,0.01
16579,0.01,0.00,0.00,0.00
16580,0.44,0.19,0.03,0.13
16581,0.05,0.05,0.25,0.03


In [14]:
df_numeric.loc[invalid_index] = df_numeric_

In [15]:
df_numeric = df_numeric.astype(float)

In [16]:
df_numeric.loc[invalid_index]

,NA_Sales,EU_Sales,JP_Sales,Other_Sales
10,0.48,0.33,0.00,0.06
36,0.30,0.03,0.04,0.03
45,0.06,0.08,0.16,0.03
64,0.01,0.00,0.00,0.00
73,0.22,0.06,0.00,0.01
...,...,...,...,...
16510,0.01,0.00,0.00,0.00
16556,0.00,0.00,0.02,0.00
16563,0.32,0.22,0.00,0.04
16565,0.00,0.03,0.00,0.00


In [17]:
df_ =df.copy()

In [18]:
df_.loc[:,"NA_Sales":	"Other_Sales"] = df_numeric

# 4. 정정한 데이터 저장

In [19]:
drive.mount('/content/drive')
path = '/content/drive/My Drive/dataset/vgames2_final.csv'

with open(path, 'w') as f:
  df_.to_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
df2 = pd.read_csv("/content/drive/My Drive/dataset/vgames2_final.csv").drop(axis=1, columns="Unnamed: 0")
df2

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0.00,0.00,0.00
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0.00,0.00,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.00,0.00,0.02,0.00
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0.00,0.00,0.00
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0.00,0.04
...,...,...,...,...,...,...,...,...,...
16578,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0.00,0.01
16579,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0.00,0.00,0.00
16580,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16581,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03
